# GWAS
 - **Project:** GP2 Parkinson's disease meta-GWAS on European, Ashkenazi Jewish, Finnish, and Icelandic individuals 
 - **Versions:** Python/3.10
 - **Last Updated:** November 2024 

### Notebook Overview
Use PoPS (polygenic priority score) to prioritize genes at GWAS loci.

### CHANGELOG
* 06-MAR-2025: Notebook cleaned 
* 31-OCT-2024: Notebook started

# Set it up.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import os
from google.colab import drive
from scipy.stats import zscore

! ls -lsth

Uncompress the tar.gzs that have been uploaded - these are big files take ~3hr 

In [ ]:
! tar -xzvf *.tar.gz

# Run PoPS

In [ ]:
%%bash
python ./pops-master/pops.py \
--gene_annot_path ./pops_features_pathway_naive_FUMA_compatible/gene_annot.txt \
--feature_mat_prefix ./pops_features_pathway_naive_FUMA_compatible/munged_features/pops_features \
--num_feature_chunks 99 --magma_prefix magma \
--control_features ./pops_features_pathway_naive_FUMA_compatible/control.features --out_prefix pops_run


/content/drive/.shortcut-targets-by-id/1pVR1aJbwsTOVw865BA45-vBesxdGS3UJ/GP2_META/FLAMES/./pops-master/pops.py:94: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  gene_annot_df = pd.read_csv(gene_annot_path, delim_whitespace=True).set_index("ENSGID")
/content/drive/.shortcut-targets-by-id/1pVR1aJbwsTOVw865BA45-vBesxdGS3UJ/GP2_META/FLAMES/./pops-master/pops.py:101: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  magma_df = pd.read_csv(magma_prefix + ".genes.out", delim_whitespace=True)


# Now merge with MAGMA outputs.

In [ ]:
pops_df = pd.read_csv("pops_run.preds", sep="\t")
pops_positives_df = pops_df[pops_df['PoPS_Score'] > 0]
pops_positives_df.describe()

,PoPS_Score,Y,Y_proj
count,9218.000000,9123.000000,9123.000000
mean,0.169208,1.252397,0.538638
std,0.138365,1.512785,1.510464
min,0.000058,-3.466900,-4.346082
25%,0.067506,0.258920,-0.444119
50%,0.141847,1.071900,0.367599
75%,0.237745,1.994550,1.281452
max,2.680602,14.933000,14.374410


In [ ]:
magma_df = pd.read_csv("magma.genes.out", sep="\t")
magma_significant_df = magma_df[magma_df['P'] <= (0.05/18650)]
magma_significant_df.describe()

,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P
count,380.000000,3.800000e+02,3.800000e+02,380.000000,380.000000,380.0,380.000000,3.800000e+02
mean,10.494737,6.118345e+07,6.126550e+07,204.410526,14.952632,1835938.0,5.803971,3.611899e-07
std,6.340178,5.204097e+07,5.206585e+07,504.450865,22.985894,0.0,1.120736,6.346264e-07
min,1.000000,6.995370e+05,7.644280e+05,1.000000,1.000000,1835938.0,4.554000,1.000000e-50
25%,4.000000,2.843919e+07,2.845641e+07,20.000000,5.000000,1835938.0,4.908875,8.675375e-11
50%,11.000000,4.246110e+07,4.252384e+07,64.500000,9.000000,1835938.0,5.530100,1.600600e-08
75%,17.000000,9.581760e+07,9.583588e+07,188.250000,17.000000,1835938.0,6.383200,4.581075e-07
max,22.000000,2.436515e+08,2.440144e+08,5829.000000,224.000000,1835938.0,14.933000,2.632200e-06


In [ ]:
table_proto_df = pops_positives_df.merge(magma_significant_df, left_on='ENSGID', right_on='GENE')
table_proto_df.describe()
percentile_90 = table_proto_df['PoPS_Score'].quantile(0.90)
table_reduced_df = table_proto_df[table_proto_df['PoPS_Score'] >= percentile_90]
table_reduced_df.describe()
table_reduced_df.to_csv("table_reduced_df.csv")

The cell above made the short plot for display.
A larger plot of all positive PoPs scores and all MAGMA results is built bellow for the supplemental.

In [ ]:
sup_table_df = pops_positives_df.merge(magma_df, left_on='ENSGID', right_on='GENE')
sup_table_df.to_csv("sup_table_df.csv")